<a href="https://colab.research.google.com/github/ameyas1/CartPole_OpenAI/blob/master/Cart_pole.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# install dependancies, takes around 45 seconds

Rendering Dependancies



In [1]:
!pip install tf-nightly-gpu
import tensorflow as tf

    100% |████████████████████████████████| 367.1MB 65kB/s 
    100% |████████████████████████████████| 409kB 13.0MB/s 
    100% |████████████████████████████████| 3.0MB 9.4MB/s 
    100% |████████████████████████████████| 61kB 23.0MB/s 


In [0]:
#remove " > /dev/null 2>&1" to see what is going on under the hood
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

Dependancies

In [3]:
!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install ez_setup > /dev/null 2>&1
!pip install gym[atari] > /dev/null 2>&1

Requirement already up-to-date: setuptools in /usr/local/lib/python3.6/dist-packages (40.8.0)


# Imports and Helper functions


In [0]:
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) #error only
import tensorflow as tf
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML

from IPython import display as ipythondisplay
from collections import Counter

In [5]:
from tensorflow.keras.models import Sequential


from tensorflow.keras.layers import Dense, Dropout
from keras.utils import to_categorical
from tensorflow.keras.models import load_model

Using TensorFlow backend.


In [6]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

<Display cmd_param=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1001'] cmd=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1001'] oserror=None return_code=None stdout="None" stderr="None" timeout_happened=False>

In [0]:
"""
Utility functions to enable video recording of gym environment and displaying it
To enable video, just do "env = wrap_env(env)""
"""

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

# CartPole


In [0]:
# env = wrap_env(gym.make("CartPole-v0"))
env=gym.make('CartPole-v0')



In [9]:
#check out the pacman action space!
print(env.action_space)

Discrete(2)


In [0]:
# def basic_policy(obs):
#   angle=obs[2]
#   return 0 if angle<0 else 1

In [0]:
# totals=[]
# ob=[]

In [0]:

# for episode in range(10):
#   episode_rewards=0
#   obs=env.reset()
  
#   action=1
#   while True:
#     action=basic_policy(obs) 
    
#     env.render()
    
#     obs,reward,done,info=env.step(action)
#     episode_rewards+=reward
#     if done:
#       ob.append(obs)
#       show_video()
      
#       totals.append(episode_rewards)
#       break
  


# env.close()






In [12]:
env.reset()

array([-0.00142326, -0.01021911,  0.01816266, -0.01171904])

In [0]:
def training():
  
  training_data=[]
  scores=[]
  accepted_scores=[]
  for _ in range(10000):
    
    score=0
    game_memory=[]
    prev_observation=[]
    for _ in range(500):
      action=env.action_space.sample()
      observation,reward,done,info=env.step(action)
      
      if(len(prev_observation))>0:
        game_memory.append([prev_observation,action])
        
      prev_observation=observation
      score+=reward
      if done:
        break
    env.reset()
    if(score>=50):
      accepted_scores.append(score)
      for data in game_memory:
        if data[1]==1:
          output=np.array([0,1])
        else:
          output=np.array([1,0])
        training_data.append([data[0],output])
    scores.append(score)
  
  training_data_save=np.array(training_data)
  np.save('saved',training_data_save)
  
  print('Average accepted score: ',np.mean(accepted_scores))
  print('Median accepted score: ',np.median(accepted_scores))
  print(Counter(accepted_scores))
  
  
  return training_data 

In [0]:
def nn_model(input_size):
  classifier=Sequential()
  classifier.add(Dense(64,input_shape=(input_size,),activation='relu'))
  
  classifier.add(Dense(128,activation='relu'))
  classifier.add(Dropout(0.4))
  classifier.add(Dense(64,activation='relu'))
  classifier.add(Dropout(0.4))
  classifier.add(Dense(32,activation='relu'))
  classifier.add(Dropout(0.4))
  classifier.add(Dense(16,activation='relu'))
  classifier.add(Dropout(0.4))
  classifier.add(Dense(2,activation='softmax'))
  classifier.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
  
  return classifier
  
  
  
  

In [0]:
def train_model(train,model=False):
  X=[i[0] for i in train]
  X=np.array(X)
  y=[i[1] for i in train]
  y=np.array(y)
  
  
  
  if not model:
    model_1=nn_model(len(X[0]))
  else:
    model_1=load_model(model)
    
  model_1.summary()
    
  model_1.fit(X,y,epochs=3)
  return model_1
  
  

In [0]:
def game(model_cp):
  
  
  scores=[]
  choices=[]
  for each_game in range(10):
    env=wrap_env(gym.make('CartPole-v0'))
    env.render()
    score=0
    game_memory=[]
    prev_obs=[]
    env.reset()
    
  
    while True:
      if(len(prev_obs)==0):
        action=env.action_space.sample()
      elif(len(prev_obs)>0):
        prev_obs_n=[prev_obs]
      
      
        action=np.argmax(model_cp.predict([prev_obs_n]))
      
      choices.append(action) 
      observation,reward,done,info=env.step(action)
      prev_obs=observation
      if(action==1):
        game_memory.append([prev_obs,[0,1]])
      else:
        game_memory.append([prev_obs,[1,0]])
      score+=reward
      if done:
          env.close()
          show_video()

          scores.append(score)
          break
    
    
  print('average score',sum(scores)/len(scores))
  print('Choice 1:{}, choice 0:{}'.format(choices.count(1)/len(choices),choices.count(0)/len(choices)))
  print(scores)

  
  return game_memory

In [17]:
train_data=training()
model_cp=train_model(train_data)

W0301 10:38:35.306733 139827331032960 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1257: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Average accepted score:  62.11168831168831
Median accepted score:  57.0
Counter({50.0: 33, 52.0: 31, 51.0: 29, 53.0: 24, 55.0: 23, 57.0: 22, 54.0: 20, 58.0: 14, 63.0: 13, 64.0: 13, 56.0: 12, 68.0: 11, 61.0: 10, 73.0: 9, 66.0: 9, 59.0: 9, 67.0: 7, 83.0: 7, 60.0: 7, 62.0: 7, 72.0: 6, 70.0: 6, 76.0: 5, 69.0: 5, 71.0: 5, 65.0: 5, 74.0: 4, 78.0: 3, 79.0: 3, 89.0: 3, 84.0: 3, 81.0: 3, 87.0: 2, 93.0: 2, 77.0: 2, 96.0: 2, 85.0: 2, 86.0: 2, 80.0: 1, 97.0: 1, 94.0: 1, 92.0: 1, 88.0: 1, 118.0: 1, 163.0: 1, 103.0: 1, 82.0: 1, 154.0: 1, 111.0: 1, 106.0: 1})
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                320       
_________________________________________________________________
dense_1 (Dense)              (None, 128)               8320      
_________________________________________________________________
dropout (Dropout)            (Non

In [18]:

game_memory=game(model_cp)
              

average score 194.3
Choice 1:0.49871332990221307, choice 0:0.5012866700977869
[200.0, 200.0, 200.0, 200.0, 200.0, 143.0, 200.0, 200.0, 200.0, 200.0]


In [0]:
more_data=[]
for data in game_memory:
  more_data.append([data[0],np.array(data[1])])

In [0]:
model_cp.save('model_cp1.h5')

In [21]:
model_cp1=train_model(train=more_data,model='model_cp1.h5')

W0301 10:39:23.359535 139827331032960 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0301 10:39:23.363289 139827331032960 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 64)                320       
_________________________________________________________________
dense_1 (Dense)              (None, 128)               8320      
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 32)                2080      
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0

In [22]:

game_memory=game(model_cp1)

average score 200.0
Choice 1:0.498, choice 0:0.502
[200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0]
